# SQL 的五十道練習：初學者友善的資料庫八週專班

> 第三天：參考解答

[數聚點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

## 練習題指引

- 在每份練習題的開始，都會先將所有學習資料庫載入環境。
- 因此 SQL 可以指定所有學習資料庫中的資料表，不需要額外指定資料庫。
- 在 SQL 語法起點與 SQL 語法終點這兩個單行註解之間撰寫能夠得到預期結果的 SQL。
- 可以先在自己電腦的 SQLiteStudio 或者 DBeaver 寫出跟預期結果相同的 SQL 後再複製貼上到練習題。
- 執行測試的方式為點選上方選單的 Kernel -> Restart & Run All -> Restart and Run All Cells。
- 可以每寫一題就執行測試，也可以全部寫完再執行測試。
- 練習題閒置超過 10 分鐘會自動斷線，這時只要重新點選練習題連結即可重新啟動。

In [1]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd

conn = sqlite3.connect('../databases/twElection2022.db')
conn.execute("""ATTACH '../databases/kaggleSurvey2022.db' AS kaggleSurvey2022""")

## 01. 從 `kaggleSurvey2022` 資料庫查詢問題 "What programming languages do you use on a regular basis? (Select all that apply)" 的填答個數，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(15, 3) 的查詢結果。

```
                                          description      choice  n_response
0   What programming languages do you use on a reg...      Python       18653
1   What programming languages do you use on a reg...         SQL        9620
2   What programming languages do you use on a reg...           R        4571
3   What programming languages do you use on a reg...         C++        4549
4   What programming languages do you use on a reg...        Java        3862
5   What programming languages do you use on a reg...           C        3801
6   What programming languages do you use on a reg...  Javascript        3489
7   What programming languages do you use on a reg...      MATLAB        2441
8   What programming languages do you use on a reg...        Bash        1674
9   What programming languages do you use on a reg...          C#        1473
10  What programming languages do you use on a reg...         PHP        1443
11  What programming languages do you use on a reg...       Other        1342
12  What programming languages do you use on a reg...          Go         322
13  What programming languages do you use on a reg...       Julia         296
14  What programming languages do you use on a reg...        None         256
```

In [2]:
find_programming_language_selections =\
"""
-- SQL 查詢語法起點
SELECT questions.description,
       responses.choice,
       COUNT( * ) AS n_response
  FROM questions
       JOIN
       responses ON questions.id = responses.question_id
 WHERE questions.description LIKE '%programming languages%'
 GROUP BY responses.choice
 ORDER BY n_response DESC;
-- SQL 查詢語法終點
"""

## 02. 從 `kaggleSurvey2022` 資料庫查詢問題 "Select the title most similar to your current role (or most recent title if retired):" 的填答比例，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(15, 4) 的查詢結果。

```
                                          description  \
0   Select the title most similar to your current ...   
1   Select the title most similar to your current ...   
2   Select the title most similar to your current ...   
3   Select the title most similar to your current ...   
4   Select the title most similar to your current ...   
5   Select the title most similar to your current ...   
6   Select the title most similar to your current ...   
7   Select the title most similar to your current ...   
8   Select the title most similar to your current ...   
9   Select the title most similar to your current ...   
10  Select the title most similar to your current ...   
11  Select the title most similar to your current ...   
12  Select the title most similar to your current ...   
13  Select the title most similar to your current ...   
14  Select the title most similar to your current ...   

                                               choice  n_response  \
0                                      Data Scientist        1929   
1   Data Analyst (Business, Marketing, Financial, ...        1538   
2                              Currently not employed        1432   
3                                   Software Engineer         980   
4                                 Teacher / professor         833   
5   Manager (Program, Project, Operations, Executi...         832   
6                                               Other         754   
7                                  Research Scientist         593   
8                    Machine Learning/ MLops Engineer         571   
9                             Engineer (non-software)         465   
10                                      Data Engineer         352   
11                                       Statistician         125   
12                                     Data Architect          95   
13                                 Data Administrator          70   
14                                 Developer Advocate          61   

   response_percentage  
0               18.15%  
1               14.47%  
2               13.47%  
3                9.22%  
4                7.84%  
5                7.83%  
6                7.09%  
7                5.58%  
8                5.37%  
9                4.37%  
10               3.31%  
11               1.18%  
12               0.89%  
13               0.66%  
14               0.57%
```

In [3]:
find_title_percentages =\
"""
-- SQL 查詢語法起點
SELECT questions.description,
       responses.choice,
       COUNT( * ) AS n_response,
       ROUND(COUNT( * ) * 100.0 / (
                                      SELECT COUNT( * ) 
                                        FROM responses
                                       WHERE question_id = (
                                                               SELECT questions.id
                                                                 FROM questions
                                                                WHERE description LIKE '%title%'
                                                           )
                                  ), 2) || '%' AS response_percentage
  FROM questions
       JOIN
       responses ON questions.id = responses.question_id
 WHERE questions.description LIKE '%title%'
 GROUP BY responses.choice
 ORDER BY n_response DESC;
-- SQL 查詢語法終點
"""

## 03. 從 `twElection2022` 資料庫查詢同名同姓的候選人資訊，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(11, 4) 的查詢結果。

```
      id candidate  type  party
0     12       許淑華   縣市長  中國國民黨
1    132       洪志明  縣市議員  中國國民黨
2    457       李柏毅  縣市議員  中國國民黨
3    620       洪志明  縣市議員  台灣民眾黨
4    670       張志豪  縣市議員  台灣民眾黨
5    713       陳志明  縣市議員   時代力量
6    812       李柏毅  縣市議員  民主進步黨
7   1047       張志豪  縣市議員  民主進步黨
8   1094       許淑華  縣市議員  民主進步黨
9   1212       陳志明  縣市議員      無
10  1539       陳志明  縣市議員      無
```

In [4]:
find_candidates_with_same_name =\
"""
-- SQL 查詢語法起點
SELECT candidates.id,
       candidates.candidate,
       candidates.type,
       parties.party
  FROM candidates
       JOIN
       parties ON candidates.party_id = parties.id
 WHERE candidates.candidate IN (
           SELECT candidate
             FROM candidates
            GROUP BY candidate
           HAVING COUNT( * ) > 1
       )
 ORDER BY parties.id;
-- SQL 查詢語法終點
"""

## 04. 從 `twElection2022` 資料庫查詢政黨的縣市長得票率，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(11, 2) 的查詢結果。

```
      party votes_percentage
0     中國國民黨           50.14%
1     民主進步黨           41.57%
2         無            6.15%
3     台灣民眾黨            1.47%
4      時代力量            0.61%
5   台灣動物保護黨            0.02%
6      台灣維新            0.02%
7       天一黨            0.01%
8       共和黨             0.0%
9       台澎黨             0.0%
10       龍黨             0.0%
```

In [5]:
find_party_percentages_of_mayor =\
"""
-- SQL 查詢語法起點
SELECT parties.party,
       ROUND(SUM(mayors.votes) * 100.0 / (
                                             SELECT SUM(votes) 
                                               FROM mayors
                                         ), 2) || '%' AS votes_percentage
  FROM mayors
       JOIN
       candidates ON mayors.candidate_id = candidates.id
       JOIN
       parties ON candidates.party_id = parties.id
 GROUP BY parties.party
 ORDER BY SUM(mayors.votes) DESC;
-- SQL 查詢語法終點
"""

## 05. 從 `twElection2022` 資料庫查詢政黨的縣市長與縣市議員得票率，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(38, 3) 的查詢結果。

```
   campaign        party votes_percentage
0       縣市長        中國國民黨           50.14%
1       縣市長        民主進步黨           41.57%
2       縣市長            無            6.15%
3       縣市長        台灣民眾黨            1.47%
4       縣市長         時代力量            0.61%
5       縣市長      台灣動物保護黨            0.02%
6       縣市長         台灣維新            0.02%
7       縣市長          天一黨            0.01%
8       縣市長          共和黨             0.0%
9       縣市長          台澎黨             0.0%
10      縣市長           龍黨             0.0%
11     縣市議員        中國國民黨           37.75%
12     縣市議員        民主進步黨           33.28%
13     縣市議員            無           19.69%
14     縣市議員        台灣民眾黨             4.0%
15     縣市議員         時代力量            1.56%
16     縣市議員         台灣基進            1.09%
17     縣市議員       無黨團結聯盟            0.95%
18     縣市議員    小民參政歐巴桑聯盟            0.36%
19     縣市議員       台灣團結聯盟            0.33%
20     縣市議員           新黨             0.3%
21     縣市議員        社會民主黨            0.25%
22     縣市議員           綠黨            0.12%
23     縣市議員          親民黨             0.1%
24     縣市議員          勞動黨            0.06%
25     縣市議員      台灣動物保護黨            0.04%
26     縣市議員         正神名黨            0.03%
27     縣市議員         台灣維新            0.02%
28     縣市議員      中華統一促進黨            0.01%
29     縣市議員         左翼聯盟            0.01%
30     縣市議員        金色力量黨            0.01%
31     縣市議員      中國和平統一黨             0.0%
32     縣市議員  中國國家社會主義勞工黨             0.0%
33     縣市議員          共和黨             0.0%
34     縣市議員          前進黨             0.0%
35     縣市議員        台灣君民黨             0.0%
36     縣市議員       台灣新住民黨             0.0%
37     縣市議員      臺灣人民共產黨             0.0%
```

In [6]:
find_party_percentages =\
"""
-- SQL 查詢語法起點
SELECT campaign,
       party,
       votes_percentage || '%' AS votes_percentage
  FROM (
           SELECT '縣市長' AS campaign,
                  parties.party,
                  ROUND(SUM(mayors.votes) * 100.0 / (
                                                        SELECT SUM(votes) 
                                                          FROM mayors
                                                    ), 2) AS votes_percentage
             FROM mayors
                  JOIN
                  candidates ON mayors.candidate_id = candidates.id
                  JOIN
                  parties ON candidates.party_id = parties.id
            GROUP BY parties.party
           UNION
           SELECT '縣市議員' AS campaign,
                  parties.party,
                  ROUND(SUM(councillors.votes) * 100.0 / (
                                                             SELECT SUM(votes) 
                                                               FROM councillors
                                                         ), 2) AS votes_percentage
             FROM councillors
                  JOIN
                  candidates ON councillors.candidate_id = candidates.id
                  JOIN
                  parties ON candidates.party_id = parties.id
            GROUP BY parties.party
            ORDER BY campaign DESC,
                     votes_percentage DESC
       )
       AS percentage_without_sign;
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All -> Restart and Run All Cells.

In [7]:
class TestDayThreeExercises(unittest.TestCase):
    def test_01_find_programming_language_selections(self):
        programming_language_selections = pd.read_sql(find_programming_language_selections, conn)
        self.assertEqual(programming_language_selections.shape, (15, 3))
        column_values = programming_language_selections.iloc[:, 1].values
        self.assertTrue('Python' in column_values)
        self.assertTrue('SQL' in column_values)
        column_values = programming_language_selections.iloc[:, 2].values
        self.assertEqual(column_values.sum(), 57792)
    def test_02_find_title_percentages(self):
        title_percentages = pd.read_sql(find_title_percentages, conn)
        self.assertEqual(title_percentages.shape, (15, 4))
        column_values = title_percentages.iloc[:, 1].values
        self.assertTrue('Data Scientist' in column_values)
        self.assertTrue('Software Engineer' in column_values)
        column_values = title_percentages.iloc[:, 2].values
        self.assertEqual(column_values.sum(), 10630)
    def test_03_find_candidates_with_same_name(self):
        candidates_with_same_name = pd.read_sql(find_candidates_with_same_name, conn)
        self.assertEqual(candidates_with_same_name.shape, (11, 4))
        column_values = candidates_with_same_name.iloc[:, 1].values
        self.assertTrue('陳志明' in column_values)
        self.assertTrue('許淑華' in column_values)
        self.assertTrue('張志豪' in column_values)
        self.assertTrue('李柏毅' in column_values)
        self.assertTrue('洪志明' in column_values)
    def test_04_find_party_percentages_of_mayor(self):
        party_percentages_of_mayor = pd.read_sql(find_party_percentages_of_mayor, conn)
        self.assertEqual(party_percentages_of_mayor.shape, (11, 2))
        column_values = party_percentages_of_mayor.iloc[:, 0].values
        self.assertTrue('民主進步黨' in column_values)
        self.assertTrue('中國國民黨' in column_values)
        self.assertTrue('無' in column_values)
    def test_05_find_party_percentages(self):
        party_percentages = pd.read_sql(find_party_percentages, conn)
        self.assertEqual(party_percentages.shape, (38, 3))
        column_values = party_percentages.iloc[:, 1].values
        self.assertTrue('民主進步黨' in column_values)
        self.assertTrue('台灣基進' in column_values)
        self.assertTrue('小民參政歐巴桑聯盟' in column_values)
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestDayThreeExercises)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)

test_01_find_programming_language_selections (__main__.TestDayThreeExercises) ... ok
test_02_find_title_percentages (__main__.TestDayThreeExercises) ... ok
test_03_find_candidates_with_same_name (__main__.TestDayThreeExercises) ... ok
test_04_find_party_percentages_of_mayor (__main__.TestDayThreeExercises) ... ok
test_05_find_party_percentages (__main__.TestDayThreeExercises) ... ok

----------------------------------------------------------------------
Ran 5 tests in 1.019s

OK


In [8]:
print("您在「專班第三天」章節中的 {} 道 SQL 練習答對了 {} 題。".format(number_of_test_runs, number_of_successes))

您在「專班第三天」章節中的 5 道 SQL 練習答對了 5 題。
